In [644]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
# Set random seed for reproducibility
np.random.seed(0)
torch.manual_seed(123)

In [645]:
dataset_file = 'train.csv'
dataset_file_target = 'train_target.csv'
data = pd.read_csv(dataset_file)
target = pd.read_csv(dataset_file_target)
data = pd.concat([data, target], axis=1)
data = data.set_index('id')

In [646]:
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2470 entries, 0 to 2469
Columns: 235 entries, agent_1_feat_Possession% to category
dtypes: float64(212), int64(23)
memory usage: 4.4 MB


In [647]:
data=data.dropna(axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2470 entries, 0 to 2469
Columns: 223 entries, agent_1_feat_Possession% to category
dtypes: float64(200), int64(23)
memory usage: 4.2 MB


In [648]:
x_train, x_test, y_train, y_test = train_test_split(data.drop('category', axis=1), data['category'], test_size=0.2, random_state=223)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [649]:
x_tr = torch.tensor(x_train, dtype=torch.float32)
y_tr = torch.tensor(y_train.values, dtype=torch.float32)
x_t = torch.tensor(x_test, dtype=torch.float32)
y_t = torch.tensor(y_test.values, dtype=torch.float32)

In [650]:
class LogisticRegression(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.linear(x)
        out = self.sigmoid(out)
        return out

In [651]:
input_dim = x_tr.shape[1]
model = LogisticRegression(input_dim)
criterion = nn.BCELoss()  
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [652]:
y_tr.size()

torch.Size([1976])

In [653]:
num_epochs = 10000
for epoch in range(num_epochs):
    outputs = model(x_tr)
    loss = criterion(outputs.squeeze(), y_tr)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 1000 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1000/10000], Loss: 0.6903
Epoch [2000/10000], Loss: 0.6769
Epoch [3000/10000], Loss: 0.6696
Epoch [4000/10000], Loss: 0.6650
Epoch [5000/10000], Loss: 0.6619
Epoch [6000/10000], Loss: 0.6597
Epoch [7000/10000], Loss: 0.6580
Epoch [8000/10000], Loss: 0.6567
Epoch [9000/10000], Loss: 0.6557
Epoch [10000/10000], Loss: 0.6549


In [654]:
print(x_t.size())
print(y_t.size())

torch.Size([494, 222])
torch.Size([494])


In [655]:
with torch.no_grad():
    predicted = model(x_t)
    predicted = (predicted > 0.5).float()  # Convert to binary predictions (0 or 1)
    
    accuracy = (predicted.squeeze() == y_t).sum().item()/y_t.size(0)
    print(accuracy)

0.5526315789473685


# For kaggle 

In [656]:
dataset_file_kaggle = 'test.csv'
test = pd.read_csv(dataset_file_kaggle)
test.info()
test=test.dropna(axis=1)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Columns: 234 entries, agent_1_feat_Possession% to agent_2_featboth_scored_mean
dtypes: float64(212), int64(22)
memory usage: 1.0 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Columns: 222 entries, agent_1_feat_Possession% to agent_2_featboth_scored_mean_3
dtypes: float64(200), int64(22)
memory usage: 988.7 KB


In [657]:
x_train_kaggle = data.drop('category', axis=1)
y_train_kaggle = data['category']
print(x_train_kaggle.info())
print(y_train_kaggle.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2470 entries, 0 to 2469
Columns: 222 entries, agent_1_feat_Possession% to agent_2_featboth_scored_mean_3
dtypes: float64(200), int64(22)
memory usage: 4.2 MB
None
<class 'pandas.core.series.Series'>
Int64Index: 2470 entries, 0 to 2469
Series name: category
Non-Null Count  Dtype
--------------  -----
2470 non-null   int64
dtypes: int64(1)
memory usage: 38.6 KB
None


In [658]:
x_train_kaggle = scaler.fit_transform(x_train_kaggle)
test = scaler.transform(test)

In [659]:
x_tr_kaggle = torch.tensor(x_train_kaggle, dtype=torch.float32)
y_tr_kaggle = torch.tensor(y_train_kaggle.values, dtype=torch.float32)
x_t_kaggle = torch.tensor(test, dtype=torch.float32)

In [660]:
input_dim = x_tr_kaggle.shape[1]
model = LogisticRegression(input_dim)
criterion = nn.BCELoss()  
optimizer = optim.SGD(model.parameters(), lr=0.001)
num_epochs = 10000
for epoch in range(num_epochs):
    outputs = model(x_tr_kaggle)
    loss = criterion(outputs.squeeze(), y_tr_kaggle)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 1000 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1000/10000], Loss: 0.6899
Epoch [2000/10000], Loss: 0.6778
Epoch [3000/10000], Loss: 0.6713
Epoch [4000/10000], Loss: 0.6672
Epoch [5000/10000], Loss: 0.6645
Epoch [6000/10000], Loss: 0.6627
Epoch [7000/10000], Loss: 0.6613
Epoch [8000/10000], Loss: 0.6603
Epoch [9000/10000], Loss: 0.6595
Epoch [10000/10000], Loss: 0.6588


In [661]:
with torch.no_grad():
    predicted = model(x_t_kaggle)
    predicted = (predicted > 0.5).float()  # Convert to binary predictions (0 or 1)

In [662]:
print(predicted.size())
predicted = predicted.squeeze()
print(predicted.size())

torch.Size([570, 1])
torch.Size([570])


In [663]:
df = pd.DataFrame({'id': range(len(predicted)), 'category': predicted})
df['category'] = df['category'].astype(int)

In [664]:
df

,id,category
0,0,0
1,1,0
2,2,1
3,3,0
4,4,0
...,...,...
565,565,1
566,566,0
567,567,0
568,568,1


In [665]:
df.to_csv('res5.csv', index=False,header=1)